## Run Program By Pressing - >>

### Note -- Program  send's too many consequtive requests from same IP address in short period of time, So the server may refuse your connection. If that happend pls pls Restart and clear output and Restart and Rerun the Kernal. Program Will run fine
ValueError: arrays must all be same length


# References
1. Professor's Lecture and Project Discussion code. , Maximum part of my doubts were cleared here.
2. https://www.w3schools.com/python/python_dictionaries.asp ,In case i got confused or got stuck on using dictionary. 
3. https://stackoverflow.com/questions/49794808/write-list-columnwise-in-csv-file-using-python ,used some idea from here to save data into my csv file.

### Import all the necessary library's. 

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests

### And by pass the SSL certificate

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

### Creating a user agent

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

## Function to get all the Movie Names

Grab all the 500 movie names using the h3 tag from the web page one by one and return the names

In [4]:
def get_movie_names():
    name = movie.find('h3').text
    return name

## Function to get all the director Names

Pass on the details link and the movie name.
Parse the detailed link page using beautiful soup
Each page has a table which contains directors names enclosed in a summary tag
With in that tag directors name is enclosed in a anchor tag. Program strips out the director name enclosed with in the anchor tag and returns the director name. This will be done for all 500 movies

In [5]:
def get_director(movie_name,detail_link):
    # scrape the page using soup
    url = detail_link
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # for some movie director name is present
    # and for some director name is not present
    director_table = soup.find('table', class_ = 'credits', summary = movie_name + "'s Director Credits")
    if director_table is None:
        director_table = soup.find('table', class_ = 'credits', summary = movie_name + "' Director Credits")

    director_line = director_table.find_all('td', class_ = 'person')
    
    if director_line is None:
        return
    
    # once we get the class. we can strip out the director name from the class
    directors = list()
    for line in director_line:
        hyperlink = line.find('a')
        director = hyperlink.get_text()
        directors.append(director.strip())

    return directors


## Function to get all the Cast Names

Inside the detail link page there are information about cast and primary cast available. Both these together form the total cast of the movie.

In [6]:
# function get the main cast of the movie
def get_cast(detail_link):
    url = detail_link
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "'s Cast Credits")
    # check if the main cast is present or no
    if cast_table is None:
        cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "' Cast Credits")

    if cast_table is None:
        return None
    else:
        cast_line = cast_table.find_all('td', class_ = 'person')
        casts = list()
        for line in cast_line:
            hyperlink = line.find('a')
            cast = hyperlink.get_text()
            casts.append(cast.strip())
        return casts


In [7]:
# get the principal cast of movie
def p_cast(detail_link):
    url = detail_link
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    # principal cast
    p_cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "'s Principal Cast Credits")
    if p_cast_table is None:
        p_cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "' Principal Cast Credits")
        
    # check if principal cast of movie are present or no    
    if p_cast_table is None:
        return None
    else:
        p_cast_line = p_cast_table.find_all('td', class_ = 'person')
        p_casts = list()
        for line in p_cast_line:
            hyperlink = line.find('a')
            p_cast = hyperlink.get_text()
            p_casts.append(p_cast.strip())
        return p_casts


## Function to grab all 5 pages containing 500 movies

In [8]:
container = []
for i in range(5):  #0-4, each iteration will give you 100 movies
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    hundred = soup.find_all('td', class_ = 'clamp-summary-wrap') #this hundred is a list containing the 100 movies on each page
    container.extend(hundred)  #container is a list, hundred is also a list, so we use extend
#     print(container)  # container has 500 pages
print(len(container)) # now our container should have the tag for all 500 movies

500


# Main Function that runs over 5 links. asking the web page for 500 movie information.

## Note --  Program will sending too many consequtive requests from same IP address in short period of time, So the server may refuse your connection. If that happend pls pls Restart and clear output and Restart and Rerun the Kernal. Program Will run fine

In [9]:
#in order to make sure our movie name, director, and cast matches, we need to extract all 3 in one go
#this is the proper solution

count = 0

casts = []

# create a list of movies, director and casts
movies = []
directors = []
all_cast = []
movie_detail_link = []



#loop through each of the 500 movies
for movie in container:
    #use your get_name() function
    movie_name = get_movie_names()  # makes a call to function to grab 500 movie name and append it in a list
    movies.append(movie_name)


    # function gets detailed link of 500 movies
    #because we already have the tag, we can extract name directly
    movie_link_tag = movie.find('a', class_ = 'title') #then lets get movie link
    movie_link = movie_link_tag.get('href', None) # we are only dealing with one movie at a time
    movie_link = 'https://www.metacritic.com'+ movie_link
    movie_detail_link.append(movie_link)
    
    detail_page = requests.get(movie_link, headers = headers)
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")
    div = detail_soup('div', class_= 'see_all pad_btm1')
    for line in div:
        hyperlink = line.find('a')
        detail_link = hyperlink.get('href', None) 
        detail_link = 'https://www.metacritic.com'+ detail_link 
        #now we have the detail link for each movie, again, we are only dealing with one movie here

        
#         here we get director name, and append it to a list
        director = get_director(movie_name,detail_link)  
        
        directors.append(director)

        # here we get the cast and pricipal cast list
        cast = get_cast(detail_link) #now use your get_cast() function


        principal_cast = p_cast(detail_link)



        # here we get 500 casts. one by one and append all together in one final cast list
        
        
        if cast is None:
            casts = principal_cast
        elif principal_cast is None:
            casts = cast
        else:
            casts = cast + principal_cast
            
        all_cast.append(casts)
        


### prints out how many movie, director and cast got appended

In [10]:
print(len(movies))
print(len(directors))
print(len(all_cast))
print(len(movie_detail_link))

500
500
500
500


## Appending Movie Inforamtion to a file CSV

In [11]:
import pandas as pd

list1 = movies
list2 = movie_detail_link
list3 = directors
list4 = all_cast


list_ThreeToStr = [','.join(ele) for ele in list3]

list_FourToStr = []
for ele in list4:
    if ele is None:
        list_FourToStr.append(None)
    else:
        list_FourToStr.append(','.join(ele))
    
pd.DataFrame({'Movie Name': list1, 'Movie Detail Link': list2, 'Directors': list_ThreeToStr, 'Casts':list_FourToStr}).to_csv('file.csv', index=False)





# Adding data into database

In [12]:
import sqlite3

In [13]:
conn = sqlite3.connect('moviesDemo.sqlite') #this opens the music database, if it does not exist, it will create a new one
#it can also connect to remote databases, but we are just doing it on our local server
cur = conn.cursor() #cursor reads the database, like open() when we are dealing with files

In [14]:
cur.execute('DROP TABLE IF EXISTS Movies') #note we put SQL command between quotation marks '' 
cur.execute('CREATE TABLE IF NOT EXISTS Movies (movie_name TEXT, movie_link TEXT, director TEXT, primary_cast TEXT)') #CREATE TABLE + {table name} + (Column1 Constraint, Column2 Constraint...)
conn.commit()

In [15]:
list1 = movies
list2 = movie_detail_link
list3 = directors
list4 = all_cast
list_ThreeToStr = [''.join(ele) for ele in list3]

list_FourToStr = []
for ele in list4:
    if ele is None:
        list_FourToStr.append(None)
    else:
        list_FourToStr.append(''.join(ele))

for i in range(len(movies)):
    cur.execute('INSERT INTO Movies (movie_name, movie_link, director, primary_cast) VALUES (?, ?, ?, ?)', (list1[i], list2[i], list_ThreeToStr[i], list_FourToStr[i]))     

### Printing Info from database

In [16]:
print("Movies") 
cur.execute('SELECT * FROM Movies')
for row in cur:
    print(row) 
conn.commit()
conn.close()

Movies
('Citizen Kane', 'https://www.metacritic.com/movie/citizen-kane', 'Orson Welles', 'Agnes MooreheadDorothy ComingoreErskine SanfordEverett SloaneFortunio BonanovaGeorge CoulourisGeorgia BackusGus SchillingHarry ShannonPaul StewartPhilip Van ZandtRay CollinsRuth WarrickWilliam AllandJoseph CottenOrson Welles')
('The Godfather', 'https://www.metacritic.com/movie/the-godfather', 'Francis Ford Coppola', 'Abe VigodaAl LettieriDiane KeatonGianni RussoJames CaanJohn CazaleJohn MarleyRichard ConteRichard S. CastellanoRobert DuvallRudy BondSterling HaydenTalia ShireAl PacinoMarlon Brando')
('Rear Window', 'https://www.metacritic.com/movie/rear-window', 'Alfred Hitchcock', 'Havis DavenportIrene WinstonJerry AntesJesslyn FaxRand HarperFrank CadyGeorgine DarcyGrace KellyJames StewartJudith EvelynRaymond BurrRoss BagdasarianSara BernerThelma RitterWendell Corey')
('Casablanca', 'https://www.metacritic.com/movie/casablanca', 'Michael Curtiz', 'Claude RainsConrad VeidtCurt BoisDooley WilsonEnri

# Creating a dictionary where movie name is the Key of the dictinary and value are the director name and cast

In [17]:
Empty_list = []


movie_director = dict()
all_movies = []


for i in range(500):
    director_cast = dict()
    
    director = directors[i]
    cast = all_cast[i]
    movie_name = movies[i]
    
    # it first checks if movie name is present in dictionary
    # if not there then it will append the dictionary to a list
    if movie_name not in movie_director:
    
        director_cast[tuple(director)] = cast

        movie_director[movie_name] = director_cast

        all_movies.append(movie_director)
    
    # if the movie name is present then it will add II and III in front of movie name then append it to the list
    else: 
        director_cast[tuple(director)] = cast
        movie_director[movie_name+'s'] = director_cast
        if movie_name == 'Little Women' and movie_name in movie_director:
            if 'Little Women-II'in movie_director:
                director_cast[tuple(director)] = cast
                movie_director[movie_name+'-III'] = director_cast
            else:
                director_cast[tuple(director)] = cast
                movie_director[movie_name+'-II'] = director_cast
        else:
            director_cast[tuple(director)] = cast
            movie_director[movie_name+'-II'] = director_cast
   
        all_movies.append(movie_director)
    
    
print(len(all_movies))

500


# Extracting 2 piece of Information

Here user can enter the movie information he wants to extract out of 500 movies. 

### Select Movie name from here

In [18]:
print(movies)

['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Lady Vanishes', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Quo Vadis, Aida?', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Nashville', 'Ratatouille', 'Parasite', 'Rocks', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)', 'Mean Streets', 'The Lady Eve',

In [19]:
while True:
    inp_movie = input('Which movie do you want to check? >')
    
    if inp_movie in movies:
        
        
        while True:
            inp = input('What information about this movie do you want to check? (Choose director or cast) >')
            inp = inp.lower()

            if inp == 'cast' or inp == 'director':
                for movie_dic in all_movies:
                    # grab the director and the cast
                    for movie_key, movie_value in movie_dic.items():
                        if inp_movie == movie_key:
                            if inp == 'director':
                                for sub_key, sub_value in movie_value.items():
                                    dir = ''
                                    for i in sub_key: #remember sub_key is a tuple
                                        dir = dir + i + ', '

                            elif inp =='cast':
                                for sub_key, sub_value in movie_value.items():
                                    cas = ''
                                    for i in sub_value: #remember sub_value is a list
                                        cas = cas + i + ', '


                if inp == 'director': 
                    print('The director of the movie', inp_movie, 'is', dir)
                    break
                elif inp == 'cast':
                    print('The cast of the movie', inp_movie, 'includes', cas)
                    break
            else:
                print("\n")
                print("Check spelling of cast and director")
        break
    else:
        print("Movie Name invalid, Movie Name should be exactly like in the above list then Try again")

Which movie do you want to check? >Citizen Kane
What information about this movie do you want to check? (Choose director or cast) >cast
The cast of the movie Citizen Kane includes Agnes Moorehead, Dorothy Comingore, Erskine Sanford, Everett Sloane, Fortunio Bonanova, George Coulouris, Georgia Backus, Gus Schilling, Harry Shannon, Paul Stewart, Philip Van Zandt, Ray Collins, Ruth Warrick, William Alland, Joseph Cotten, Orson Welles, 


# First Question

Extract a list of director and cast and store it

In [20]:
all_directors = tuple()
all_movie_cast = []

for i in range(len(all_movies[0])):
    sub = list(movie_director.values())[i] #so this extracts the sub dictionary
    sub_key = list(sub.keys())
    sub_values = list(sub.values())
    for tpl in sub_key:
        all_directors += tpl
    for lst in sub_values:
        if lst is not None:
            all_movie_cast += lst

## Analyse how many time each director appeared in a list of 500 movies

In [21]:
all_director_lst = list(all_directors)
dictor_count = dict()
for i in all_director_lst:
    dictor_count[i] = dictor_count.get(i, 0) + 1  # go through all the director in the dict and keep a count of them
print(dictor_count)

{'Orson Welles': 5, 'Francis Ford Coppola': 5, 'Alfred Hitchcock': 12, 'Michael Curtiz': 4, 'Richard Linklater': 3, 'Krzysztof Kieslowski': 2, 'Gene Kelly': 1, 'Stanley Donen': 1, 'Charles Chaplin': 2, 'Barry Jenkins': 2, 'D.W. Griffith': 1, 'Ben Sharpsteen': 4, 'Bill Roberts': 4, 'Hamilton Luske': 2, 'Jack Kinney': 2, 'Norman Ferguson': 3, 'T. Hee': 2, 'Wilfred Jackson': 4, 'John Huston': 5, 'Guillermo del Toro': 2, 'Billy Wilder': 3, 'Steve James': 3, 'Akira Kurosawa': 2, 'Joseph L. Mankiewicz': 1, 'François Truffaut': 2, 'Sam Peckinpah': 1, 'Jim Sheridan': 1, 'Carol Reed': 2, 'Stanley Kubrick': 5, 'Jasmila Zbanic': 1, 'George Cukor': 6, 'Sam Wood': 1, 'Victor Fleming': 1, 'Cristian Mungiu': 1, 'Sergei M. Eisenstein': 1, 'Elia Kazan': 2, 'George Lucas': 2, 'John Elliotte': 1, 'Samuel Armstrong': 3, 'Alfonso Cuarón': 3, 'Ernst Lubitsch': 1, 'Sidney Lumet': 3, 'Kenneth Lonergan': 1, 'Roman Polanski': 2, 'Steve McQueen': 4, 'Charles Burnett': 1, 'Robert Altman': 4, 'Brad Bird': 2, 'Joon

## Analyse how many time each actor/actress appeared in a list of 500 movies

In [22]:
dict_count = dict()
for i in all_movie_cast:
    dict_count[i] = dict_count.get(i, 0) + 1  # go through all the movie in the dict and keep a count of them
print(dict_count)

{'Agnes Moorehead': 2, 'Dorothy Comingore': 1, 'Erskine Sanford': 2, 'Everett Sloane': 1, 'Fortunio Bonanova': 2, 'George Coulouris': 1, 'Georgia Backus': 3, 'Gus Schilling': 1, 'Harry Shannon': 2, 'Paul Stewart': 2, 'Philip Van Zandt': 1, 'Ray Collins': 4, 'Ruth Warrick': 1, 'William Alland': 1, 'Joseph Cotten': 5, 'Orson Welles': 6, 'Abe Vigoda': 1, 'Al Lettieri': 1, 'Diane Keaton': 3, 'Gianni Russo': 1, 'James Caan': 3, 'John Cazale': 3, 'John Marley': 2, 'Richard Conte': 1, 'Richard S. Castellano': 1, 'Robert Duvall': 5, 'Rudy Bond': 4, 'Sterling Hayden': 4, 'Talia Shire': 2, 'Al Pacino': 5, 'Marlon Brando': 7, 'Havis Davenport': 1, 'Irene Winston': 1, 'Jerry Antes': 1, 'Jesslyn Fax': 1, 'Rand Harper': 1, 'Frank Cady': 1, 'Georgine Darcy': 1, 'Grace Kelly': 2, 'James Stewart': 6, 'Judith Evelyn': 1, 'Raymond Burr': 1, 'Ross Bagdasarian': 1, 'Sara Berner': 1, 'Thelma Ritter': 2, 'Wendell Corey': 1, 'Claude Rains': 3, 'Conrad Veidt': 1, 'Curt Bois': 1, 'Dooley Wilson': 1, 'Enrique Ac

# Second Question

In [23]:
## Program will create a dictionary of cast that have worked with the director and keep a count of it.

In [24]:
def cast_counting(user_director):
    director_cast_dict = all_movies[0]
    dire = user_director
    
    
    # Extract Each Director and cast from the All movie and seperate the one which user inputs
    list_of_common_dir = []
    for i in director_cast_dict:
        # create list of director
        for j in director_cast_dict[i]:
            if dire == j[0]:
                list_of_common_dir.append(director_cast_dict[i])

                
                
                
    # Seperate the cast from the dictionary
    list_of_cast_with_dir = []
    for i in list_of_common_dir:
        for key, val in i.items():
            my_key = key[0]
            list_of_cast_with_dir.append(val)

    # append every cast in a list
    count_cast = []
    for bigList in list_of_cast_with_dir:
        if bigList is None:
            continue
        for j in bigList:
            count_cast.append(j)
            
    # Calculate the count
    freq = {}
    for item in count_cast:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
            
    dic = {dire : freq}
    
    
    return dic

## List of all directors inorder to select

In [25]:
all_directors = tuple()
all_movie_cast = []

for i in range(len(all_movies[0])):
    sub = list(movie_director.values())[i] #so this extracts the sub dictionary
    sub_key = list(sub.keys())
    sub_values = list(sub.values())
    for tpl in sub_key:
        all_directors += tpl
    for lst in sub_values:
        if lst is not None:
            all_movie_cast += lst

In [26]:
all_director_lst = list(all_directors)
dictor_count = dict()
for i in all_director_lst:
    dictor_count[i] = dictor_count.get(i, 0) + 1  # go through all the director in the dict and keep a count of them
print(dictor_count)

{'Orson Welles': 5, 'Francis Ford Coppola': 5, 'Alfred Hitchcock': 12, 'Michael Curtiz': 4, 'Richard Linklater': 3, 'Krzysztof Kieslowski': 2, 'Gene Kelly': 1, 'Stanley Donen': 1, 'Charles Chaplin': 2, 'Barry Jenkins': 2, 'D.W. Griffith': 1, 'Ben Sharpsteen': 4, 'Bill Roberts': 4, 'Hamilton Luske': 2, 'Jack Kinney': 2, 'Norman Ferguson': 3, 'T. Hee': 2, 'Wilfred Jackson': 4, 'John Huston': 5, 'Guillermo del Toro': 2, 'Billy Wilder': 3, 'Steve James': 3, 'Akira Kurosawa': 2, 'Joseph L. Mankiewicz': 1, 'François Truffaut': 2, 'Sam Peckinpah': 1, 'Jim Sheridan': 1, 'Carol Reed': 2, 'Stanley Kubrick': 5, 'Jasmila Zbanic': 1, 'George Cukor': 6, 'Sam Wood': 1, 'Victor Fleming': 1, 'Cristian Mungiu': 1, 'Sergei M. Eisenstein': 1, 'Elia Kazan': 2, 'George Lucas': 2, 'John Elliotte': 1, 'Samuel Armstrong': 3, 'Alfonso Cuarón': 3, 'Ernst Lubitsch': 1, 'Sidney Lumet': 3, 'Kenneth Lonergan': 1, 'Roman Polanski': 2, 'Steve McQueen': 4, 'Charles Burnett': 1, 'Robert Altman': 4, 'Brad Bird': 2, 'Joon

In [27]:
# Alfred Hitchcock
# Orson Welles
# Francis Ford Coppola
# Michael Curtiz
# Peter Weir
# Michel Gondry

dire_1 = input("Enter First director name: ")
dire_2 = input("Enter Second director name: ")

dire_1_one = 0
dire_2_one = 0

for i in directors:
    if i[0] == dire_1:
        dire_1_one = 1
        
for i in directors:
    if i[0] == dire_2:
        dire_2_one = 1
        
# print(dire_2_one)

while True: 
    if dire_1 != '' and dire_2 != '' and dire_1_one == 1 and dire_2_one == 1:
        break
    else:
        print("\n")
        print("Enter a valid movie name")
        if dire_1_one != 1:
            print(" Your Director 1 name was wrong: Try again")
            dire_1 = input("Enter First director name: ")
            for i in directors:
                if i[0] == dire_1:
                    dire_1_one = 1
        if dire_2_one != 1:
            print(" Your Director 2 name was wrong. try again")
            dire_2 = input("Enter Second director name: ") 
            for i in directors:
                if i[0] == dire_2:
                    dire_2_one = 1


# if my director is present in my list brea
        
dic1 = cast_counting(dire_1)
print(dic1)
dic2 = cast_counting(dire_2)
print(dic2)

       


Enter First director name: Peter Weir
Enter Second director name: Michel Gondry
{'Peter Weir': {'Angel Schmiedt': 1, 'Blair Slater': 1, 'Brian Delate': 1, 'Don Taylor': 1, 'Fritz Dominique': 1, 'Heidi Schanz': 1, 'Holland Taylor': 1, 'Judy Clayton': 1, 'Laura Linney': 1, 'Natascha McElhone': 1, 'Noah Emmerich': 1, 'Peter Krause': 1, 'Ron Taylor': 1, 'Ted Raymond': 1, 'Jim Carrey': 1}}
{'Michel Gondry': {'Amir Ali Said': 1, 'Brian Price': 1, 'David Cross': 1, 'Debbon Ayer': 1, 'Elijah Wood': 1, 'Gerry Robert Byrne': 1, 'Jane Adams': 1, 'Kirsten Dunst': 1, 'Mark Ruffalo': 1, 'Paulie Litt': 1, 'Ryan Whitney': 1, 'Thomas Jay Ryan': 1, 'Jim Carrey': 1, 'Kate Winslet': 1, 'Tom Wilkinson': 1}}


#### Append the dictionary that was returned into a common list

In [28]:
all_movie = []
all_movie.append(dic1)
all_movie.append(dic2)
print(all_movie)

[{'Peter Weir': {'Angel Schmiedt': 1, 'Blair Slater': 1, 'Brian Delate': 1, 'Don Taylor': 1, 'Fritz Dominique': 1, 'Heidi Schanz': 1, 'Holland Taylor': 1, 'Judy Clayton': 1, 'Laura Linney': 1, 'Natascha McElhone': 1, 'Noah Emmerich': 1, 'Peter Krause': 1, 'Ron Taylor': 1, 'Ted Raymond': 1, 'Jim Carrey': 1}}, {'Michel Gondry': {'Amir Ali Said': 1, 'Brian Price': 1, 'David Cross': 1, 'Debbon Ayer': 1, 'Elijah Wood': 1, 'Gerry Robert Byrne': 1, 'Jane Adams': 1, 'Kirsten Dunst': 1, 'Mark Ruffalo': 1, 'Paulie Litt': 1, 'Ryan Whitney': 1, 'Thomas Jay Ryan': 1, 'Jim Carrey': 1, 'Kate Winslet': 1, 'Tom Wilkinson': 1}}]


#### we first need to extract the list of actors and how many time they have worked together

In [29]:
for i in all_movie:
    for k, v in i.items():
        if k == dire_1:     # if the director name matches as that of user input grab the cast
            sub_dic1 = v

for i in all_movie:
    for k, v in i.items():
        if k == dire_2:
            sub_dic2 = v
            
# print(sub_dic1)
# print(sub_dic2)

### then program creates the combined list containing all actors that the 2 directors have worked with

In [30]:
#first program extract the actor names, which are the keys
sub_actor1 = list(sub_dic1.keys())
sub_actor2 = list(sub_dic2.keys())

#then program convert them into sets, so that it can find a combined set using the union function
set_actor1 = set(sub_actor1)
set_actor2 = set(sub_actor2)
combine_set = set_actor1.union(set_actor2)

# print(combine_set)

In [31]:
#let's next create a dictionary out of the combined list, filling in 0 for all the values
dir1_actor = {}
for i in combine_set:
    dir1_actor[i] = 0
print(dir1_actor)

dir2_actor = {}
for i in combine_set:
    dir2_actor[i] = 0
print(dir2_actor)

{'Tom Wilkinson': 0, 'Laura Linney': 0, 'Noah Emmerich': 0, 'Paulie Litt': 0, 'Elijah Wood': 0, 'Thomas Jay Ryan': 0, 'Ron Taylor': 0, 'David Cross': 0, 'Gerry Robert Byrne': 0, 'Blair Slater': 0, 'Heidi Schanz': 0, 'Debbon Ayer': 0, 'Mark Ruffalo': 0, 'Peter Krause': 0, 'Jim Carrey': 0, 'Angel Schmiedt': 0, 'Judy Clayton': 0, 'Jane Adams': 0, 'Don Taylor': 0, 'Ryan Whitney': 0, 'Holland Taylor': 0, 'Ted Raymond': 0, 'Kirsten Dunst': 0, 'Brian Delate': 0, 'Kate Winslet': 0, 'Fritz Dominique': 0, 'Natascha McElhone': 0, 'Brian Price': 0, 'Amir Ali Said': 0}
{'Tom Wilkinson': 0, 'Laura Linney': 0, 'Noah Emmerich': 0, 'Paulie Litt': 0, 'Elijah Wood': 0, 'Thomas Jay Ryan': 0, 'Ron Taylor': 0, 'David Cross': 0, 'Gerry Robert Byrne': 0, 'Blair Slater': 0, 'Heidi Schanz': 0, 'Debbon Ayer': 0, 'Mark Ruffalo': 0, 'Peter Krause': 0, 'Jim Carrey': 0, 'Angel Schmiedt': 0, 'Judy Clayton': 0, 'Jane Adams': 0, 'Don Taylor': 0, 'Ryan Whitney': 0, 'Holland Taylor': 0, 'Ted Raymond': 0, 'Kirsten Dunst':

In [32]:
#for those actors that director1 have worked with, replace the 0 with the actual values
for key, value in dir1_actor.items():
    for k, v in sub_dic1.items():
        if key == k:
            dir1_actor[key] = v
print(dir1_actor)

{'Tom Wilkinson': 0, 'Laura Linney': 1, 'Noah Emmerich': 1, 'Paulie Litt': 0, 'Elijah Wood': 0, 'Thomas Jay Ryan': 0, 'Ron Taylor': 1, 'David Cross': 0, 'Gerry Robert Byrne': 0, 'Blair Slater': 1, 'Heidi Schanz': 1, 'Debbon Ayer': 0, 'Mark Ruffalo': 0, 'Peter Krause': 1, 'Jim Carrey': 1, 'Angel Schmiedt': 1, 'Judy Clayton': 1, 'Jane Adams': 0, 'Don Taylor': 1, 'Ryan Whitney': 0, 'Holland Taylor': 1, 'Ted Raymond': 1, 'Kirsten Dunst': 0, 'Brian Delate': 1, 'Kate Winslet': 0, 'Fritz Dominique': 1, 'Natascha McElhone': 1, 'Brian Price': 0, 'Amir Ali Said': 0}


In [33]:
#do the same thing for director2
for key, value in dir2_actor.items():
    for k, v in sub_dic2.items():
        if key == k:
            dir2_actor[key] = v
print(dir2_actor)

{'Tom Wilkinson': 1, 'Laura Linney': 0, 'Noah Emmerich': 0, 'Paulie Litt': 1, 'Elijah Wood': 1, 'Thomas Jay Ryan': 1, 'Ron Taylor': 0, 'David Cross': 1, 'Gerry Robert Byrne': 1, 'Blair Slater': 0, 'Heidi Schanz': 0, 'Debbon Ayer': 1, 'Mark Ruffalo': 1, 'Peter Krause': 0, 'Jim Carrey': 1, 'Angel Schmiedt': 0, 'Judy Clayton': 0, 'Jane Adams': 1, 'Don Taylor': 0, 'Ryan Whitney': 1, 'Holland Taylor': 0, 'Ted Raymond': 0, 'Kirsten Dunst': 1, 'Brian Delate': 0, 'Kate Winslet': 1, 'Fritz Dominique': 0, 'Natascha McElhone': 0, 'Brian Price': 1, 'Amir Ali Said': 1}


In [34]:
#now that we have a uniform dictionary across the 2 directors, we can extract the values in vectors
#ready for comparison
dir1_vector = list(dir1_actor.values())
dir2_vector = list(dir2_actor.values())
print(dir1_vector)
print(dir2_vector)

[0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0]
[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1]


### calculate cosine similarity

In [35]:
divident = 0 
for i in range(len(dir1_vector)):
    summing = dir1_vector[i] * dir2_vector[i]
    divident = divident + summing
print(divident)

1


In [36]:
a = 0
b = 0
for i in range(len(dir1_vector)):
    sum1 = (dir1_vector[i])**2
    a = a + sum1
    sum2 = (dir2_vector[i])**2
    b = b + sum2
divisor = (a**0.5)*(b**0.5)
print(divisor)

15.000000000000002


In [37]:
cosine_similarity = divident/divisor
print("The Cosine similarity of 2 directors are: ",cosine_similarity)

The Cosine similarity of 2 directors are:  0.06666666666666665


# Third Question

### Here the program creates a dictionary of all actor/actresses that have worked with one another

In [38]:
def actor_counting(director_cast_dict,actor):
        
    # Extract Each Director and cast from the All movie and seperate the one which user inputs
    list_of_all_casts = []
    for i in director_cast_dict:

        for key,value in director_cast_dict[i].items():
            list_of_all_casts.append(value)

    
    # grab a list of cast with that specific actor in it
    list_of_cast_with_actor = []
    for i in list_of_all_casts:
        if i is None:
            continue
    
        if actor in i:
            list_of_cast_with_actor.append(i)
             
            
            
    # remove that actor from the list
    list_of_cast_without_actor = []
    for lst in list_of_cast_with_actor:
        for j in lst:
            if j != actor:
                list_of_cast_without_actor.append(j)

    # count how many times each one have appeared in this list
    frequ = {}
    for item in list_of_cast_without_actor:
        if (item in frequ):
            frequ[item] += 1
        else:
            frequ[item] = 1
            
            
    dic = {actor : frequ}
    
    return dic

## Callling the function here. Checking Out the famous Actor each one had worked with.

In [39]:
import operator
director_cast_dict = all_movies[0]

# created a list of 5 actors

# l1 = ["Orson Welles","Agnes Moorehead","Joseph Cotten","Ray Collins",'Erskine Sanford']
# l1 = ["Orson Welles","Agnes Moorehead","Joseph Cotten","Ray Collins",'Lady Gaga']
# l1 = ["Orson Welles","Agnes Moorehead","Joseph Cotten","Bradley Cooper",'Lady Gaga']
l1 = ["Orson Welles","Agnes Moorehead","Ray Collins","Bradley Cooper",'Lady Gaga']

five_actor = []

# looping through the list so as to compare each actor with one another
# eg 1 with 2,3,4,5 and 2 with 3,4,5 and so on
for i in range(len(l1)-1):
    actor_one= l1[i]
    for j in range(i+1,len(l1)):
        actor_two = l1[j]
        l2 = [actor_one,actor_two]
        five_actor.append(l2)

cos = []
all_movie = []
for i in five_actor:
#     print(i)
    actor_one = i[0]
    actor_two = i[1]
    
    dic_one = actor_counting(director_cast_dict,actor_one)
    
    print("actor one is: ",actor_one)
#     print("\n")
#     print(dic_one)
#     print("\n\n")
    
    list_of_dict_values_one = list(dic_one.values())
    for i in list_of_dict_values_one:
        print("The Famous Actore",actor_one,"has worked with is: ",max(i.items(), key=operator.itemgetter(1))[0])
        
    print("\n")
    
    dic_two = actor_counting(director_cast_dict,actor_two)
    print("actor two is: ",actor_two)
#     print("\n")
#     print(dic_two)
#     print("\n")
    
    list_of_dict_values_two = list(dic_two.values())
    for i in list_of_dict_values_two:
        print("The Famous Actore",actor_two,"has worked with is: ",max(i.items(), key=operator.itemgetter(1))[0])
    print("\n")
       
    all_movie.append(dic_one)
    all_movie.append(dic_two)
#     print(all_movie)
    
    #we first need to extract the list of actors and how many time they have worked together
    for i in all_movie:
        for k, v in i.items():
            if k == actor_one:
                sub_dic1 = v

    for i in all_movie:
        for k, v in i.items():
            if k == actor_two:
                sub_dic2 = v

#     print(sub_dic1)
#     print(sub_dic2)


    #then we need to create the combined list containing all actors that the 2 actors have worked with
    #first we extract the actor names, which are the keys
    sub_actor1 = list(sub_dic1.keys())
    sub_actor2 = list(sub_dic2.keys())

    #then we convert them into sets, so that we can find a combined set using the union function
    set_actor1 = set(sub_actor1)
    set_actor2 = set(sub_actor2)
    combine_set = set_actor1.union(set_actor2)

#     print(combine_set)    
    

    #let's next create a dictionary out of the combined list, filling in 0 for all the values
    # you will see why in the next step
    dir1_actor = dict()
    for i in combine_set:
        dir1_actor[i] = 0
#     print(dir1_actor)

    dir2_actor = dict()
    for i in combine_set:
        dir2_actor[i] = 0
#     print(dir2_actor)    

    #for those actors that actor_1 have worked with, replace the 0 with the actual values
    for key, value in dir1_actor.items():
        for k, v in sub_dic1.items():
            if key == k:
                dir1_actor[key] = v
#     print(dir1_actor)

    #do the same thing for director2
    for key, value in dir2_actor.items():
        for k, v in sub_dic2.items():
            if key == k:
                dir2_actor[key] = v
#     print(dir2_actor)   
    
    #now that we have a uniform dictionary across the 2 directors, we can extract the values in vectors
    #ready for comparison
    dir1_vector = list(dir1_actor.values())
    dir2_vector = list(dir2_actor.values())
#     print(dir1_vector)
#     print(dir2_vector)

    divident = 0 
    for i in range(len(dir1_vector)):
        summing = dir1_vector[i] * dir2_vector[i]
        divident = divident + summing
#     print(divident)    
    
    a = 0
    b = 0
    for i in range(len(dir1_vector)):
        sum1 = (dir1_vector[i])**2
        a = a + sum1
        sum2 = (dir2_vector[i])**2
        b = b + sum2
    divisor = (a**0.5)*(b**0.5)
#     print(divisor)    
    
    cosine_similarity = divident/divisor
    print("\n")
    print("The cosine similarity of", actor_one ,"and", actor_two ,"are: ",cosine_similarity)
    print("\n")
    print("---------------------------------------------------------------")
    print("\n")
    cos.append(cosine_similarity)

actor one is:  Orson Welles
The Famous Actore Orson Welles has worked with is:  Joseph Cotten


actor two is:  Agnes Moorehead
The Famous Actore Agnes Moorehead has worked with is:  Erskine Sanford




The cosine similarity of Orson Welles and Agnes Moorehead are:  0.6383450536225745


---------------------------------------------------------------


actor one is:  Orson Welles
The Famous Actore Orson Welles has worked with is:  Joseph Cotten


actor two is:  Ray Collins
The Famous Actore Ray Collins has worked with is:  Orson Welles




The cosine similarity of Orson Welles and Ray Collins are:  0.5970149490461657


---------------------------------------------------------------


actor one is:  Orson Welles
The Famous Actore Orson Welles has worked with is:  Joseph Cotten


actor two is:  Bradley Cooper
The Famous Actore Bradley Cooper has worked with is:  Andrew Dice Clay




The cosine similarity of Orson Welles and Bradley Cooper are:  0.0


---------------------------------------

## The Combined Cosine Similarity are

In [40]:
print("The Cosine similarity is : ",cos)

The Cosine similarity is :  [0.6383450536225745, 0.5970149490461657, 0.0, 0.0, 0.7031374768163903, 0.0, 0.0, 0.0, 0.0, 0.8728767972360391]


## Cosine Similarity Of One actor with Remaining Actors

In [41]:
cosine_similarity_of_actor_one = cos[0] + cos[1] + cos[2] + cos[3]
cosine_similarity_of_actor_two = cos[0] + cos[4] + cos[5] + cos[6]
cosine_similarity_of_actor_three = cos[1] + cos[4] + cos[7] + cos[8]
cosine_similarity_of_actor_four = cos[2] + cos[5] + cos[7] + cos[9]
cosine_similarity_of_actor_five = cos[3] + cos[6] + cos[8] + cos[9]

print("cosine similarity of ",l1[0]," with", l1[1],",",l1[2],",",l1[3],",",l1[4],"is : ",cosine_similarity_of_actor_one)
print("cosine similarity of ",l1[1]," with",l1[0],",",l1[2],",",l1[3],",",l1[4], "is : ",cosine_similarity_of_actor_two)
print("cosine similarity of ",l1[2]," with" ,l1[0],",",l1[1],",",l1[3],",",l1[4],"is : ",cosine_similarity_of_actor_three)
print("cosine similarity of ",l1[3]," with" ,l1[0],",",l1[1],",",l1[2],",",l1[4], "is : ",cosine_similarity_of_actor_four)
print("cosine similarity of ",l1[4],"with" ,l1[0],",",l1[1],",",l1[2],",",l1[3], "is : ",cosine_similarity_of_actor_five)

cosine similarity of  Orson Welles  with Agnes Moorehead , Ray Collins , Bradley Cooper , Lady Gaga is :  1.2353600026687404
cosine similarity of  Agnes Moorehead  with Orson Welles , Ray Collins , Bradley Cooper , Lady Gaga is :  1.3414825304389648
cosine similarity of  Ray Collins  with Orson Welles , Agnes Moorehead , Bradley Cooper , Lady Gaga is :  1.300152425862556
cosine similarity of  Bradley Cooper  with Orson Welles , Agnes Moorehead , Ray Collins , Lady Gaga is :  0.8728767972360391
cosine similarity of  Lady Gaga with Orson Welles , Agnes Moorehead , Ray Collins , Bradley Cooper is :  0.8728767972360391
